In [1]:
import pandas as pd 
import os 
import numpy as np 
from scipy.spatial.distance import cdist
import folium
import re
from scipy.spatial import distance_matrix
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans


In [2]:
#selecção de dados de input
centro = '8811417' #input("Introduza o centro: ")
codigo_prestador='3720'
codigo_prestador_2='SHP0'
giros_excluir=['3700F','3720G']
cp_validation =['3720']

Importação de ficheiros da pasta FTP

In [3]:
folder_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Dados_centros")

files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]


combined_df = pd.DataFrame()


for file in files:
    file_path = os.path.join(folder_path, file)
    
    
    df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)



/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_2050/4127806100.py:13: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding="latin-1", delimiter=';')


Limpeza dos dados


In [4]:
# -- limpeza do dataset 
df = combined_df
df.columns=[x.lower().strip() for x in df.columns]
df.drop_duplicates()
df['centro'] = df['centro'].astype(str).str.replace('.0',"")
df['centro'] = df['centro'].str.strip()
df['giro'] = df['giro'].str.replace(' ', '').str.strip().str[:5]
df['cp'] = df['cp'].str.replace(' ', '').str.strip()
df['longitude'] = df['longitude'].str.replace('\n', '').str.strip()
df=df[df.latitude != '0']
df=df[df.latitude != '0.0']
df = df[df['latitude'] != '']
df=df[df.cod_t_even == 'I']
df = df[df['cp'].str.len() >= 8]

In [5]:
# remove os fins de semana 
df['data_criacao'] = pd.to_datetime(df['data_criacao'], format='%d/%m/%Y')

df = df[df['data_criacao'].dt.dayofweek < 5]

In [6]:
df=df[df['centro']==centro]


In [7]:
# -- considera apenas cp7 que ocorreram várias vezes --#

giro_unique_cp7 = df.groupby('cp')['data_criacao'].nunique().reset_index(name='N_Unique_cp7')

giro_unique_cp7 = giro_unique_cp7[giro_unique_cp7['N_Unique_cp7'] >= 2]

giro_unique_cp7=np.array(giro_unique_cp7['cp'].unique())

df = df[df['cp'].isin(giro_unique_cp7)]

In [8]:
#-- excluir cp7 efetuados por várias rotas --#

unique_routes_by_cp = df.groupby('cp')['giro'].nunique().reset_index(name='Unique_Routes_Count')
unique_routes_by_cp = unique_routes_by_cp.sort_values(by='Unique_Routes_Count', ascending=False)

unique_routes_by_cp_filtered = unique_routes_by_cp[unique_routes_by_cp['Unique_Routes_Count'] ==1]

lista_excl_cp7=np.array(unique_routes_by_cp_filtered['cp'].unique())

df = df[df['cp'].isin(lista_excl_cp7)]

cp7_unique_counts = df['cp'].value_counts()

In [9]:
# selecção de giros 

giro_unique_counts = df.groupby('giro')['data_criacao'].nunique().reset_index(name='N_Unique_Data_Criacao')

giro_unique_counts = giro_unique_counts[
    ((giro_unique_counts['giro'].str[:4] == codigo_prestador) |
     (giro_unique_counts['giro'].str[:4] == codigo_prestador_2)) &
    (~giro_unique_counts['giro'].apply(lambda x: any(x.startswith(prefix) for prefix in giros_excluir)))
]

giro_unique_counts_average = giro_unique_counts['N_Unique_Data_Criacao'].mean()
giro_unique_counts_average

giro_unique_counts=giro_unique_counts[giro_unique_counts['N_Unique_Data_Criacao'] > 0]
giro_unique_counts

print(giro_unique_counts)

lista_val_giro=np.array(giro_unique_counts['giro'].unique())

print(lista_val_giro)

df = df[df['giro'].isin(lista_val_giro)]

     giro  N_Unique_Data_Criacao
5   3720A                      7
6   3720B                      7
7   3720C                      7
8   3720D                      7
9   3720E                      7
10  3720F                      7
55  SHP08                      6
['3720A' '3720B' '3720C' '3720D' '3720E' '3720F' 'SHP08']


In [72]:
# -- validação do cp4 -- 

def expand_values(cp_values):
    expanded_values = []
    
    for value in cp_values:
        value = int(value)
        expanded_values.extend([str(value + i) for i in range(5)])
    
    return expanded_values

expanded_validation = expand_values(cp_validation)

#print(cp_counts)
print(cp_validation)
print(expanded_validation)

print(len(cp7_unique_counts))

['3720']
['3720', '3721', '3722', '3723', '3724']
3003


Dados Sig Postal

In [73]:


file_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Sig_postal")


files = [file for file in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, file))]

for file in files:

    file_path = os.path.join(file_path, file)
    df_sig = pd.read_csv(file_path, encoding="latin-1", delimiter=';')
    


In [74]:


df_sig.columns=[x.lower().strip() for x in df_sig.columns]

df_sig = df_sig.apply(lambda x: x.astype(str).str.replace(' ', '').str.strip() if pd.notnull(x.name) else x)

df_sig['cp7'] = df_sig['cp7'].astype(str)

df_sig['point_x'] = df_sig['point_x'].str.replace('\n', '').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace('\n', '').str.strip()
df_sig['point_x'] = df_sig['point_x'].str.replace('.', ',').str.strip()
df_sig['point_y'] = df_sig['point_y'].str.replace('.', ',').str.strip()
df_sig=df_sig[df_sig.point_x != '0']
df_sig=df_sig[df_sig.point_y != '0']
df_sig=df_sig[df_sig.point_x != '0.0']
df_sig=df_sig[df_sig.point_y != '0.0']
df_sig = df_sig[df_sig['point_x'] != '']
df_sig = df_sig[df_sig['point_y'] != '']
df_sig = df_sig[df_sig['point_x'] != 'nan']
df_sig = df_sig[df_sig['point_y'] != 'nan']
df_sig = df_sig.dropna()


In [75]:
df_sig = df_sig[df_sig['cp4'].isin(expanded_validation)]

In [76]:

df['data_criacao'] = pd.to_datetime(df['data_criacao'], dayfirst=True)

result = df.groupby(['cp', 'giro']).size().reset_index(name='count')

idx = result.groupby('cp')['count'].idxmax()

df_result = result.loc[idx].reset_index(drop=True)



In [77]:
merged_df = pd.merge(df_sig, df_result[['cp', 'giro']], how='left', left_on='cp7', right_on='cp')

merged_df.drop(['cp', 'cp3','cpalf'], axis=1, inplace=True)


In [78]:
# selected_columns = ['cp7', 'cp4', 'point_x', 'point_y', 'giro']
# selected_df = merged_df[selected_columns]

# downloads_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/resultados")


# file_path = os.path.join(downloads_path, f'{codigo_prestador}.xlsx')
# selected_df.to_excel(file_path, index=False)

# print(f"File saved at: {file_path}")

Preencher os cp7 sem rota 

In [79]:
# calcular a distância entre pontos e afetar os cp7 sem rota aos que tem rota mais próximos


merged_df['point_x'] = merged_df['point_x'].str.replace(',', '.').astype(float)
merged_df['point_y'] = merged_df['point_y'].str.replace(',', '.').astype(float)

def fill_giro(row):
    if pd.isna(row['giro']):
        
        non_null_rows = merged_df[merged_df['giro'].notna()]

        
        distances = cdist(np.array([[row['point_x'], row['point_y']]]), non_null_rows[['point_x', 'point_y']], metric='cityblock')

        
        closest_index = np.argmin(distances)

        
        return non_null_rows.iloc[closest_index]['giro']
    else:
        return row['giro']

merged_df['giro'] = merged_df.apply(fill_giro, axis=1)

print(merged_df)


          cp7   cp4   point_x    point_y   giro
0    3720-001  3720 -8.478426  40.835010  3720A
1    3720-002  3720 -8.427220  40.825090  3720C
2    3720-003  3720 -8.427612  40.825737  3720C
3    3720-004  3720 -8.431667  40.846447  3720C
4    3720-005  3720 -8.439014  40.843870  3720C
..        ...   ...       ...        ...    ...
776  3720-999  3720 -8.477479  40.840258  3720A
777  3723-301  3723 -8.474722  40.861929  3720D
778  3723-302  3723 -8.502784  40.871863  3720B
779  3723-303  3723 -8.482161  40.850504  3720E
780  3723-304  3723 -8.482291  40.830760  3720A

[781 rows x 5 columns]


In [80]:
from sklearn.metrics import silhouette_samples

labels = merged_df.giro
data_points = merged_df[['point_x', 'point_y']]

individual_silhouette_scores = silhouette_samples(data_points, labels)

merged_df['individual_silhouette'] = individual_silhouette_scores

print(merged_df[['cp7', 'cp4', 'giro', 'individual_silhouette']])


          cp7   cp4   giro  individual_silhouette
0    3720-001  3720  3720A               0.647231
1    3720-002  3720  3720C               0.154467
2    3720-003  3720  3720C               0.150126
3    3720-004  3720  3720C              -0.101556
4    3720-005  3720  3720C              -0.156645
..        ...   ...    ...                    ...
776  3720-999  3720  3720A               0.581148
777  3723-301  3723  3720D              -0.296839
778  3723-302  3723  3720B               0.660411
779  3723-303  3723  3720E               0.066771
780  3723-304  3723  3720A               0.575103

[781 rows x 4 columns]


In [81]:
# specific_point = merged_df[merged_df['cp7'] == '2415-741']

# print(specific_point[['cp7', 'cp4', 'giro', 'individual_silhouette']])


In [82]:
# # visualizar em mapa
# df_final=merged_df

# m = folium.Map(location=[df_final['point_y'].mean(), df_final['point_x'].mean()], zoom_start=12, tiles='OpenStreetMap')

# color_mapping = {}
# unique_rotas = df_final['giro'].unique()

# fixed_colors = ["#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD",
#           "#34495E", "#229954", "#7D3C98", "#8E44AD", "#D35400",
#           "#E74C3C", "#F39C12", "#2E86C1", "#A569BD", "#3498DB",
#           "#27AE60", "#1ABC9C", "#E67E22", "#BDC3C7", "#34495E",
#           "#16A085", "#F1C40F", "#2C3E50", "#58D68D", "#AF7AC5",
#           "#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD"]

# for index, rota in enumerate(unique_rotas):
#     color_mapping[rota] = fixed_colors[index % len(fixed_colors)]


# rota_feature_groups = {}
# for rota in unique_rotas:
#     rota_feature_groups[rota] = folium.FeatureGroup(name=rota) 

# for _, row in df_final.iterrows():
#     rota_color = color_mapping.get(row['giro'], '#1787FE')  
#     folium.RegularPolygonMarker(  
#         location=[row['point_y'], row['point_x']],
#         number_of_sides=10,  
#         radius=3,
#         popup=f"CP7: {row['cp7']}, Giro: {row['giro']}",
#         color=rota_color,
#         fill=True,
#         fill_color=rota_color
#     ).add_to(rota_feature_groups[row['giro']])  


# for rota, feature_group in rota_feature_groups.items():
#     m.add_child(feature_group)


# folium.LayerControl().add_to(m)


# m


In [83]:
#tratamento dos pontos com baixa classificação de silhueta 

giro_centers = merged_df.groupby('giro')[['point_x', 'point_y']].mean()

def assign_giro(row, giro_centers):
    point_coords = row[['point_x', 'point_y']]
    distances = distance_matrix([point_coords], giro_centers)[0]
    closest_giro = giro_centers.index[distances.argmin()]
    return closest_giro

merged_df['correct_giro'] = merged_df.apply(lambda row: assign_giro(row, giro_centers), axis=1)

silhouette_avg = silhouette_score(merged_df[['point_x', 'point_y']], merged_df['correct_giro'])
print(f"Average Silhouette Score: {silhouette_avg}")

limiar_silhueta = -0.2

points_to_reassign = merged_df[merged_df['individual_silhouette'] < limiar_silhueta]

merged_df.loc[points_to_reassign.index, 'giro'] = points_to_reassign['correct_giro']

merged_df = merged_df.drop(['correct_giro'], axis=1)

merged_df


Average Silhouette Score: 0.31506195368043494


,cp7,cp4,point_x,point_y,giro,individual_silhouette
0,3720-001,3720,-8.478426,40.835010,3720A,0.647231
1,3720-002,3720,-8.427220,40.825090,3720C,0.154467
2,3720-003,3720,-8.427612,40.825737,3720C,0.150126
3,3720-004,3720,-8.431667,40.846447,3720C,-0.101556
4,3720-005,3720,-8.439014,40.843870,3720C,-0.156645
...,...,...,...,...,...,...
776,3720-999,3720,-8.477479,40.840258,3720A,0.581148
777,3723-301,3723,-8.474722,40.861929,3720E,-0.296839
778,3723-302,3723,-8.502784,40.871863,3720B,0.660411
779,3723-303,3723,-8.482161,40.850504,3720E,0.066771


In [84]:

df_final=merged_df

m = folium.Map(location=[df_final['point_y'].mean(), df_final['point_x'].mean()], zoom_start=12, tiles='OpenStreetMap')

color_mapping = {}
unique_rotas = df_final['giro'].unique()

fixed_colors = ["#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD",
          "#34495E", "#229954", "#7D3C98", "#8E44AD", "#D35400",
          "#E74C3C", "#F39C12", "#2E86C1", "#A569BD", "#3498DB",
          "#27AE60", "#1ABC9C", "#E67E22", "#BDC3C7", "#34495E",
          "#16A085", "#F1C40F", "#2C3E50", "#58D68D", "#AF7AC5",
          "#3498DB", "#E74C3C", "#27AE60", "#F39C12", "#8E44AD"]


for index, rota in enumerate(unique_rotas):
    color_mapping[rota] = fixed_colors[index % len(fixed_colors)]


rota_feature_groups = {}
for rota in unique_rotas:
    rota_feature_groups[rota] = folium.FeatureGroup(name=rota) 

for _, row in df_final.iterrows():
    rota_color = color_mapping.get(row['giro'], '#1787FE')  
    folium.RegularPolygonMarker(  
        location=[row['point_y'], row['point_x']],
        number_of_sides=10,  
        radius=3,
        popup=f"CP7: {row['cp7']}, Giro: {row['giro']}",
        color=rota_color,
        fill=True,
        fill_color=rota_color
    ).add_to(rota_feature_groups[row['giro']])  


for rota, feature_group in rota_feature_groups.items():
    m.add_child(feature_group)


folium.LayerControl().add_to(m)


m


In [85]:
selected_columns = ['cp7', 'cp4', 'point_x', 'point_y', 'giro']
selected_df = merged_df[selected_columns]

downloads_path = os.path.join(os.path.expanduser("~"), "Downloads/Pit_stop/Resultados")

file_path = os.path.join(downloads_path, f'{codigo_prestador}.xlsx')
selected_df.to_excel(file_path, index=False)

print(f"File saved at: {file_path}")

File saved at: C:\Users\1420844\Downloads/Pit_stop/Resultados\3720.xlsx
